In [1]:
from src.data.create_dataset import *
from src.features.data_cleaning import *
import numpy as np
import pandas as pd
import itertools
import calendar

In [2]:
year = '2020'
month = '08'

In [3]:
df = read_data(year, month)

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-08-01 00:02:53,2020-08-01 00:28:54,1.0,13.20,1.0,N,237,16,2.0,36.5,3.0,0.5,0.00,0.0,0.3,40.30,2.5
1,2.0,2020-08-01 00:08:11,2020-08-01 00:16:28,1.0,2.83,1.0,N,146,137,1.0,10.5,0.5,0.5,1.50,0.0,0.3,15.80,2.5
2,2.0,2020-08-01 00:22:14,2020-08-01 00:22:20,1.0,0.00,1.0,N,264,264,2.0,2.5,0.5,0.5,0.00,0.0,0.3,3.80,0.0
3,2.0,2020-08-01 00:31:35,2020-08-01 00:36:54,1.0,2.34,1.0,N,79,141,1.0,8.0,0.5,0.5,2.36,0.0,0.3,14.16,2.5
4,2.0,2020-08-01 00:39:18,2020-08-01 00:42:09,1.0,1.32,1.0,N,141,263,1.0,5.5,0.5,0.5,1.86,0.0,0.3,11.16,2.5


In [5]:
remove_out_of_range_data(df, year, month)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
74405,2.0,2020-08-04 03:12:27,2020-08-05 03:11:03,4.0,3.22,1.0,N,179,179,2.0,14.00,0.5,0.5,0.00,0.00,0.3,15.30,0.0
74667,2.0,2020-08-04 05:08:10,2020-08-05 05:00:25,1.0,18.20,1.0,N,42,77,2.0,50.50,0.5,0.5,0.00,6.12,0.3,57.92,0.0
74906,2.0,2020-08-04 06:03:35,2020-08-05 00:00:00,1.0,20.59,1.0,N,132,26,1.0,56.50,0.0,0.5,0.00,0.00,0.3,57.30,0.0
78660,2.0,2020-08-04 09:00:40,2020-08-05 08:14:24,1.0,3.02,1.0,N,262,170,1.0,12.50,0.0,0.5,3.16,0.00,0.3,18.96,2.5
78944,2.0,2020-08-04 09:35:52,2020-08-05 08:53:45,1.0,2.33,1.0,N,68,142,1.0,9.00,0.0,0.5,1.23,0.00,0.3,13.53,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007279,NaN,2020-08-17 10:44:01,2020-08-17 11:08:05,NaN,9.97,NaN,NaN,87,42,NaN,37.98,0.0,0.5,2.75,0.00,0.3,41.53,0.0
1007280,NaN,2020-08-17 10:32:00,2020-08-17 11:14:00,NaN,16.93,NaN,NaN,209,10,NaN,58.00,0.0,0.5,2.75,0.00,0.3,61.55,0.0
1007281,NaN,2020-08-17 10:30:00,2020-08-17 10:47:00,NaN,2.75,NaN,NaN,215,121,NaN,21.07,0.0,0.5,2.75,0.00,0.3,24.62,0.0
1007282,NaN,2020-08-17 10:21:32,2020-08-17 10:54:42,NaN,10.63,NaN,NaN,212,107,NaN,31.03,0.0,0.5,2.75,6.12,0.3,40.70,0.0
